In [1]:
from kamgon import *

In [2]:
class CourseSchedulingProblem(CSPProblem):
    """
    ปัญหาการจัดตารางเรียนโดยไม่ให้เกิดการขัดแย้งด้านเวลาและทรัพยากร
    """
    
    def __init__(self, courses: List[str], time_slots: List[str], 
                 rooms: List[str], conflicts: List[tuple]):
        """
        เริ่มต้นปัญหาการจัดตารางเรียน
        
        Args:
            courses: รายการรายวิชา
            time_slots: รายการช่วงเวลา
            rooms: รายการห้องเรียน
            conflicts: รายการคู่รายวิชาที่ไม่สามารถเรียนพร้อมกันได้
        """
        self.time_slots = time_slots
        self.rooms = rooms
        self.conflicts = conflicts
        
        # สร้าง domain ที่เป็น combination ของเวลาและห้อง
        possible_slots = [(time, room) for time in time_slots for room in rooms]
        domains = {course: possible_slots.copy() for course in courses}
        
        super().__init__(courses, domains)
    
    def is_consistent(self, variable: str, value: Any, assignment: Dict[str, Any]) -> bool:
        """
        ตรวจสอบว่าการจัดรายวิชาในช่วงเวลาและห้องนี้ขัดแย้งหรือไม่
        
        Args:
            variable: ชื่อรายวิชา
            value: tuple ของ (เวลา, ห้อง)
            assignment: การจัดตารางที่มีอยู่แล้ว
            
        Returns:
            True ถ้าสามารถจัดรายวิชานี้ได้โดยไม่ขัดแย้ง
        """
        time, room = value
        
        for assigned_course, assigned_slot in assignment.items():
            if assigned_course == variable:
                continue
                
            assigned_time, assigned_room = assigned_slot
            
            # ตรวจสอบการขัดแย้งห้อง (ห้องเดียวกันในเวลาเดียวกัน)
            if time == assigned_time and room == assigned_room:
                return False
            
            # ตรวจสอบการขัดแย้งรายวิชา (รายวิชาที่ไม่ควรเรียนพร้อมกัน)
            if time == assigned_time:
                for course1, course2 in self.conflicts:
                    if ((variable == course1 and assigned_course == course2) or 
                        (variable == course2 and assigned_course == course1)):
                        return False
        
        return True

In [3]:
# การใช้งาน Course Scheduling Problem
if __name__ == "__main__":
    print("\n=== Course Scheduling Problem ===")
    
    courses = ['คณิตศาสตร์', 'ฟิสิกส์', 'เคมี', 'ชีววิทยา', 'ประวัติศาสตร์']
    time_slots = ['08:00-10:00', '10:00-12:00', '13:00-15:00', '15:00-17:00']
    rooms = ['ห้อง101', 'ห้อง102', 'ห้อง201']
    
    # รายวิชาที่ไม่ควรเรียนพร้อมกัน (นักเรียนคนเดียวกันต้องเรียน)
    conflicts = [('คณิตศาสตร์', 'ฟิสิกส์'), ('เคมี', 'ชีววิทยา')]
    
    scheduling = CourseSchedulingProblem(courses, time_slots, rooms, conflicts)
    
    # แก้ปัญหาด้วย backtracking
    solution = backtracking_search(scheduling, verbose=False)
    
    if solution:
        print("ตารางเรียนที่จัดได้:")
        for course, slot in solution.items():
            time, room = slot
            print(f"  {course}: {time} ที่ {room}")
    else:
        print("ไม่สามารถจัดตารางเรียนได้")


=== Course Scheduling Problem ===
ตารางเรียนที่จัดได้:
  คณิตศาสตร์: 08:00-10:00 ที่ ห้อง101
  ฟิสิกส์: 10:00-12:00 ที่ ห้อง101
  เคมี: 08:00-10:00 ที่ ห้อง102
  ชีววิทยา: 10:00-12:00 ที่ ห้อง102
  ประวัติศาสตร์: 08:00-10:00 ที่ ห้อง201
